## 02 - Repeated Holdout

Consideramos a unidade de tempo (nesse caso eras) e não o número de linhas.

In [1]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor

In [2]:
data = pd.read_csv('data-processed/train.csv')

## Repeated Holdout 60/10

60% dos dados são usadados para treino, esse caso de 1 a 72 (60% de 120 eras é 72 eras). E tiramos fora 10% do final (10% de 120 eras é 12).

EScolhemos um ponto entre 72 e 108 e desse ponto para traz pegamos 60% das eras como treimo e 10% pra frente desse ponto como teste.

In [3]:
results_val = []

for seed in range(20):

    np.random.seed(seed)
    split_point = np.random.randint(72, 108)
    print("Repetition {} - Split point {}".format(seed, split_point))
    
    train = data[(data['era'] > split_point - 72) & (data['era'] <= split_point)]
    val = data[(data['era'] > split_point) & (data['era'] <= split_point+12)]

    X_train = train.filter(regex=r'feature')
    X_val = val.filter(regex=r'feature')

    y_train = train['target']
    y_val = val['target']
     
    mdl = LGBMRegressor(max_depth=5, num_leaves=2**5, learning_rate=0.01, n_estimators=2000, colsample_bytree=0.1, random_state=0)
    mdl.fit(X_train, y_train)
    
    predictions = pd.Series(mdl.predict(X_val))
    ranked_predictions = predictions.rank(pct=True, method="first")
    correlation = np.corrcoef(y_val, ranked_predictions)[0, 1]
    #print(correlation)
    
    results_val.append(correlation)
    print("Correlation {}".format(correlation))
    print()

Repetition 0 - Split point 72
Correlation 0.047833804460405444

Repetition 1 - Split point 84
Correlation 0.04743417504869361

Repetition 2 - Split point 87
Correlation 0.052257727471143996

Repetition 3 - Split point 96
Correlation 0.037939659199590976

Repetition 4 - Split point 77
Correlation 0.030858306372656397

Repetition 5 - Split point 107
Correlation 0.04121782740138564

Repetition 6 - Split point 82
Correlation 0.0397508927101892

Repetition 7 - Split point 76
Correlation 0.03874572802761041

Repetition 8 - Split point 75
Correlation 0.03967148980901571

Repetition 9 - Split point 100
Correlation 0.03602350973304467

Repetition 10 - Split point 81
Correlation 0.04120797573238562

Repetition 11 - Split point 97
Correlation 0.04204479649283199

Repetition 12 - Split point 83
Correlation 0.0394582055737802

Repetition 13 - Split point 90
Correlation 0.051736449432678314

Repetition 14 - Split point 96
Correlation 0.037939659199590976

Repetition 15 - Split point 80
Correlation 0

In [4]:
np.median(results_val)

0.03971119125960246

In [5]:
np.min(results_val)

0.030858306372656397

In [6]:
np.max(results_val)

0.052257727471143996

In [7]:
np.mean(results_val)

0.041708255837289446

In [8]:
len(results_val)

20

# Fim